In [ ]:
import pandas as pd 
import numpy as np
from konlpy.tag import Kkma
import pymysql

In [ ]:
class Keyword_Extracting:
    conn= None
    curs = None
    review_cleaned_list = []
    kp = None
    num = None #포스트 1개 안의 리뷰 갯수
    
    #table 4
    post_code = None
    pkw_list = [] #post keyword_list
    pkw_count = {} #post keyword_count
    post_keyword = []
    
    #table 5
    this_review = None
    
    
    def __init__(self, postcode):
        #DB 커넥트
        self.conn = pymysql.connect(host="ppam.ceubmhz1zgkv.ap-northeast-1.rds.amazonaws.com",user = "admin",password ="ppam1234", db="ppam_test",charset="utf8")
        self.curs = self.conn.cursor(pymysql.cursors.DictCursor)
        self.post_code = postcode
        
        #datasetting
        self.kp = pd.read_sql("select keyword from keyword_pool", self.conn)
        
        idx = []
        review_table = pd.read_sql("select * from review_table", self.conn)
        for i in range(len(review_table['review_code'])):
            if review_table['review_code'][i][:len(self.post_code)] == self.post_code:
                self.review_cleaned_list.append(review_table['review_cleaned'][i])
                idx.append(review_table['review_code'][i]) #for table5
            else:
                pass
        self.num = len(self.review_cleaned_list)
        self.this_review = pd.DataFrame({'review_code':idx, 'review_cleaned':self.review_cleaned_list})
        print('~~~~~~~', self.post_code, ' 작업 시작 ~~~~~~~')
        
        
        
    def total_keyword(self): #포스트 전체 리뷰에 대한 분석
        for i in range(0, self.num): #i = 리뷰 1개
            temp = []
            find = []
            tagged = Kkma().pos(self.review_cleaned_list[i])
            
            for j in range(0, len(tagged)):
                if tagged[j][0] in self.kp['keyword'].values:
                    find.append(tagged[j][0])
            for v in find: #한 리뷰 안의 키워드 중복제거 -> temp = 리뷰 1개에 대한 키워드 모음(중복 제거)
                if v not in temp:
                    temp.append(v)
            for k in range(0, len(temp)): # pkw_list = 포스트 1개에 대한 키워드 모음집(중복 포함)
                self.pkw_list.append(temp[k])
        
          #포스트 1개에 대한 키워드 빈도수 카운트하기
        for i in self.pkw_list:
            try: self.pkw_count[i] += 1
            except: self.pkw_count[i] = 1
        self.pkw_count = sorted(self.pkw_count.items(), reverse = True, key=lambda item: item[1])
            
        #빈도수 가장 높은 5개 추리기
        del self.pkw_count[5:]
        temp2 = []
        for i in range(0, len(self.pkw_count)):
            temp2.append(self.pkw_count[i][0])
        self.post_keyword = temp2
        print(self.post_keyword , "를 table4에 업로드 예정")
        
        
        
        
    def insert_to_table4(self): #db table4에 데이터 업로드
        if len(self.post_keyword) == 5:
            sql = '''INSERT INTO ppam_test.keyword_table (post_code, keyword1, keyword2, keyword3, keyword4, keyword5) VALUES(%s,%s,%s,%s,%s,%s);'''
            self.curs.execute(sql, (self.post_code, self.post_keyword[0], self.post_keyword[1], self.post_keyword[2], self.post_keyword[3], self.post_keyword[4]))
            self.conn.commit()
        elif len(self.post_keyword) == 4:
            sql = '''INSERT INTO ppam_test.keyword_table (post_code, keyword1, keyword2, keyword3, keyword4) VALUES(%s,%s,%s,%s,%s);'''
            self.curs.execute(sql, (self.post_code, self.post_keyword[0], self.post_keyword[1], self.post_keyword[2], self.post_keyword[3]))
            self.conn.commit()
        elif len(self.post_keyword) == 3:
            sql = '''INSERT INTO ppam_test.keyword_table (post_code, keyword1, keyword2, keyword3) VALUES(%s,%s,%s,%s);'''
            self.curs.execute(sql, (self.post_code, self.post_keyword[0], self.post_keyword[1], self.post_keyword[2]))
            self.conn.commit()
        elif len(self.post_keyword) == 2:
            sql = '''INSERT INTO ppam_test.keyword_table (post_code, keyword1, keyword2) VALUES(%s,%s,%s);'''
            self.curs.execute(sql, (self.post_code, self.post_keyword[0], self.post_keyword[1]))
            self.conn.commit()
        elif len(self.post_keyword) == 1:
            sql = '''INSERT INTO ppam_test.keyword_table (post_code, keyword1) VALUES(%s,%s);'''
            self.curs.execute(sql, (self.post_code, self.post_keyword[0]))
            self.conn.commit()
        elif len(self.post_keyword) == 0:
            sql = '''INSERT INTO ppam_test.keyword_table (post_code) VALUES(%s);'''
            self.curs.execute(sql, (self.post_code))
            self.conn.commit()
            
        print(self.post_keyword , "를 table4에 업로드 완료")
        
        
        
        ############### 여기서부터는 table 5 ##################
        
    def keywords_from_review(self, review):
        temp = []
        find = []
        tagged = Kkma().pos(review)
        
        for j in range(0, len(tagged)):
            if tagged[j][0] in self.kp['keyword'].values:
                find.append(tagged[j][0])
        for v in find:
            if v not in temp:
                temp.append(v)
        return temp
    
    
    def list_for_table5(self, before): #before = post_keyword랑 비교할 리뷰 1개분의 키워드 리스트, 아마 keywords_from_review의 temp
        result = [0,0,0,0,0] #있으면 1, 없으면 0인 리스트
        
        for i in range(0, 5):
            if self.post_keyword[i] in before:
                result[i]=1
        return result

    
    
    def repeat(self):
        time = len(self.this_review['review_code'])
        review_code = None
        review_keyword = []
        
        for i in range(0, time): #포스트 1개 안의 리뷰 수 만큼 반복
            review_code = None
            review_keyword = None
            result = []
            
            review_code = self.this_review['review_code'][i]
            review_keyword = self.keywords_from_review(self.this_review['review_cleaned'][i])
            result = self.list_for_table5(review_keyword)
            
            #insert to table5
            sql = '''INSERT INTO ppam_test.review_keyword_table2 (review_code, keyword1, keyword2, keyword3, keyword4, keyword5) VALUES(%s,%s,%s,%s,%s,%s);'''
            self.curs.execute(sql, (review_code, result[0], result[1], result[2], result[3], result[4]))
            self.conn.commit()
            print(review_code, ' : ', review_keyword, )

        
        
        
    def doit(self):
        if self.num < 20:
            print("리뷰가 없다")
        else:
            self.total_keyword()
            self.insert_to_table4()
            self.repeat()
        


In [ ]:
conn = pymysql.connect(host="ppam.ceubmhz1zgkv.ap-northeast-1.rds.amazonaws.com",user = "admin",password ="ppam1234", db="ppam_test",charset="utf8")
curs = conn.cursor(pymysql.cursors.DictCursor)
df = pd.read_sql("select post_code from post_table", conn)

df

In [ ]:
for i in range(0, len(df['post_code'])):
    a = Keyword_Extracting(df[post_code][i])
    a.doit()